# Notebook for plotting

Before you can run this notebook you should run our model. You can find the run_model script in the `scripts/run_on_cluster` folder.



In [2]:
import pickle
import sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import datetime
sys.path.append("../")
sys.path.append("../covid19_inference_repo")

import covid19_uefa
import covid19_inference as cov19


# Matplotlib config
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams["figure.figsize"] = [3.4, 2.7]  # APS single column
matplotlib.rcParams["figure.dpi"] = 300  # this primarily affects the size on screen
#matplotlib.rcParams['axes.linewidth'] = 0.3
matplotlib.rcParams["axes.labelcolor"] = "black"
matplotlib.rcParams["axes.edgecolor"] = "black"
matplotlib.rcParams["xtick.color"] = "black"
matplotlib.rcParams["ytick.color"] = "black"
matplotlib.rcParams["xtick.labelsize"] = 8
matplotlib.rcParams["ytick.labelsize"] = 8
matplotlib.rcParams["axes.labelsize"] = 8
matplotlib.rcParams["axes.titlesize"]= 10
matplotlib.rcParams["legend.fontsize"] = 6
matplotlib.rcParams["legend.title_fontsize"] = 8


colors_paul = ["#233954",
"#ea5e48",
"#1e7d72",
"#f49546",
"#e8bf58", # dark
"#5886be",
"#f3a093",
"#53d8c9",
"#f2da9c",
"#f9c192", # light
] 


# General config
fig_path = "./figures"
colors = ["tab:blue","tab:purple","tab:orange","tab:green"]
countries = ["England","Scotland","Germany","France"]
xlim_ts = [datetime.datetime(2021,5,30),datetime.datetime(2021,7,23)]

In [3]:
def get_from_trace(var,trace):
    """ Reshapes and returns an numpy array from an arviz trace
    """
    var = np.array(trace.posterior[var])
    var = var.reshape((var.shape[0]*var.shape[1],) + var.shape[2:])
    return var

def plot_casesOverview(trace,model,dl,country,xlim_fraction):
    """
    Plots an overview for the new cases i.e. cases per gender with model fit
    and the fraction between male and female.
    
    Parameters
    ----------
    trace:
        arviz trace of model run
    model:
        corresponding model
    dl:
        dataloader
    """
    
    new_cases = get_from_trace("new_cases",trace)
    fig, axes = plt.subplots(3,1,)

    ## New cases by sex
    for sex in range(2):
        cov19.plot._timeseries( # male
            x=pd.date_range(model.sim_begin,model.sim_end),
            y=new_cases[:,:,sex]/dl.population[sex,0]*1e6,
            ax=axes[sex],
            what="model",
            color=colors[sex]
        )
        cov19.plot._timeseries( # female
            x=pd.date_range(model.data_begin,model.data_end),
            y=dl.new_cases_obs[:,sex,0]/dl.population[sex,0]*1e6,
            ax=axes[sex],
            what="data",
            color=colors[sex],
            ms=2
        )
        axes[sex].set_ylabel("Daily new cases\nrelative to population")
        #axes[sex].set_ylim(0,(dl.new_cases_obs[:,:,0]/dl.population[:,0]).max()*1e6+50)
        axes[sex].set(xticklabels=[])

    ## Fraction male/female
    cov19.plot._timeseries(
        x = pd.date_range(model.sim_begin,model.sim_end),
        y = (new_cases[:,:,0]/dl.population[0,0]) / (new_cases[:,:,1]/dl.population[1,0]), # male/female
        what="model",
        ax=axes[2],
        color="tab:gray"
    )
    cov19.plot._timeseries(
        x = pd.date_range(model.data_begin,model.data_end),
        y = (dl.new_cases_obs[:,0,0]/dl.population[0,0])/(dl.new_cases_obs[:,1,0]/dl.population[1,0]), # male/female
        what="data",
        ax=axes[2],
        color="tab:gray",
        ms=2
    )
    axes[2].set_ylabel("Fraction between\nmale and female cases")
    axes[2].set_ylim(xlim_fraction)
    # Apply same ylims to all axes
    for ax in axes:
        ax.set_xlim(xlim_ts)
        for label in ax.xaxis.get_ticklabels()[::2]:
            label.set_visible(False)
    axes[1].set(ylabel=None)
    
    # Set title for figure
    fig.suptitle(country)

    # Save figure as pdf and png
    kwargs = {
        "transparent":True,
        "dpi":300,
        "bbox_inches":"tight"
    }
    fig.savefig(f"{fig_path}/fig_casesOverview_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_casesOverview_{country}.png", **kwargs)
    plt.close(fig=fig)
    
def plot_reproductionNumber(trace,model,dl,country):
    """
    Plots the base and soccer reproduction number 
    
    Parameters
    ----------
    trace:
        arviz trace of model run
    model:
        corresponding model
    dl:
        dataloader
    """
    R_base = get_from_trace("R_t_base",trace)
    R_soccer = get_from_trace("R_t_soccer",trace)
    C_base = get_from_trace("C_base",trace)
    C_soccer = get_from_trace("C_soccer",trace)
    
    # Calculate Effective R per sex
    #R_eff = np.einsum('dt,mf->dtmf',R_base, C_base) + np.einsum('dt,dmf->dtmf',R_soccer, C_soccer)
    
    fig,axes = plt.subplots(2,1)

    # Plot base and soccer Reproduction number
    cov19.plot._timeseries(
        x = pd.date_range(model.sim_begin,model.sim_end),
        y = R_base,
        what="model",
        ax=axes[0],
        color="tab:gray"
    )
    axes[0].axhline(1.0,color="tab:gray",ls="--",zorder=-5,lw=0.5)
    cov19.plot._timeseries(
        x = pd.date_range(model.sim_begin,model.sim_end),
        y = R_soccer,
        what="model",
        ax=axes[1],
        color="tab:gray"
    )
    axes[1].axhline(0,color="tab:gray",ls="--",zorder=-5,lw=0.5)
    
    
    axes[0].set_ylabel("Base\nreproduction number")
    axes[1].set_ylabel("Additive\nreproduction number")
    
    
    # Disable xticks of top axis 
    axes[0].set(xticklabels=[])
    
    # Apply same ylims to all axes
    for ax in axes:
        ax.set_xlim(xlim_ts)
        for label in ax.xaxis.get_ticklabels()[::2]:
            label.set_visible(False)
            
    # Set title for figure
    fig.suptitle(country)
    
    # Save figure as pdf and png        
    kwargs = {
        "transparent":True,
        "dpi":300,
        "bbox_inches":"tight"
    }
    fig.savefig(f"{fig_path}/fig_reproductionNumbers_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_reproductionNumbers_{country}.png", **kwargs)
    plt.close(fig=fig)
    
def plot_alpha(trace,model,dl,country,beta=True):
    """
    Plots the base and soccer reproduction number 
    
    Parameters
    ----------
    trace:
        arviz trace of model run
    model:
        corresponding model
    dl:
        dataloader
    
    TODO
    ----
    maybe put into one figure
    """
    # Plot mean alpha
    ax = covid19_uefa.plot.distribution(
        model=model,
        trace=trace,
        key="alpha_mean",
        title="Mean effect of soccer games",
        dist_math="α_{mean}",
    )
    fig = ax[0].get_figure()
    # Save figure as pdf and png        
    kwargs = {
        "transparent":True,
        "dpi":300,
        "bbox_inches":"tight"
    }
    fig.savefig(f"{fig_path}/fig_alphaMean_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_alphaMean_{country}.png", **kwargs)
    plt.close()

    # Plot delta alpha (per game)
    ax = covid19_uefa.plot.distribution(
        model=model,
        trace=trace,
        key="Delta_alpha_g_sparse",
        title="Effect for each soccer game $\Deltaα_g$",
        dist_math="\Deltaα")
    
    fig = ax[0].get_figure()
    # Save figure as pdf and png        
    kwargs = {
        "transparent":True,
        "dpi":300,
        "bbox_inches":"tight"
    }
    fig.savefig(f"{fig_path}/fig_alphaGames_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_alphaGames_{country}.png", **kwargs)
    plt.close()
    
    
    ax = covid19_uefa.plot.distribution(
        model=model,
        trace=trace,
        key="sigma_alpha_g",
        title="Error of the effect (hierachical) $\sigma^α_g$",
        dist_math="\sigma^{α}"
    )
    fig = ax[0].get_figure()
    # Save figure as pdf and png        
    kwargs = {
        "transparent":True,
        "dpi":300,
        "bbox_inches":"tight"
    }
    fig.savefig(f"{fig_path}/fig_alphaError_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_alphaError_{country}.png", **kwargs)
    plt.close()
    
import seaborn as sns
import pandas as pd
def plot_violin(trace,model,dl,country,beta=False):
    """
    Violin plot for each game and country.
    
    If beta, uses beta variable instead of alpha
    """
    key = "alpha"
    if beta:
        key = "beta"
        
    # Sanitiy check, sometime countries do not have a stadium:
    if (len(dl.beta_prior[dl.beta_prior > 0]) == 0):
        return
    
    α_mean = get_from_trace(f"{key}_mean",trace)
    σ_g = get_from_trace(f"sigma_{key}_g",trace)
    Δα_g_sparse = get_from_trace(f"Delta_{key}_g_sparse",trace)
    alpha = α_mean[:,None] + np.einsum("dg,d->dg",Δα_g_sparse,σ_g)

    nGames = alpha.shape[-1]
    # Plot alpha
    fig, ax = plt.subplots(1,1,figsize=[3.4, 2.7*nGames/4])
    sns.violinplot(
        data=alpha,
        scale="width",
        inner="quartile",
        orient="h",
        ax=ax
    )
    
    # Get date of game and participants
    games = dl.timetable.loc[(dl.alpha_prior > 0)[0]]
    if beta:
        games = dl.timetable.loc[(dl.beta_prior > 0)[0]]
    
    # Construct ylabels
    ylabels= []
    for i, vals in games.iterrows():
        label =vals["date"].strftime("%d.%-m.%y")
        label +=f'\n{vals["team1"]} vs {vals["team2"]}'
        label +=f'\nin {vals["location"]}'
        ylabels.append(label)
        
    ax.set_yticklabels(ylabels)
    #ax.set_xlabel("Effect of game")

    # Set title for figure
    fig = ax.get_figure()
    fig.suptitle(country + f" - {key}")
    
    # Save figure as pdf and png        
    kwargs = {
        "transparent":True,
        "dpi":300,
        "bbox_inches":"tight"
    }
    fig.savefig(f"{fig_path}/fig_violin{key}_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_violin{key}_{country}.png", **kwargs)
    plt.close(fig=fig)
    
def plot_contacts(trace,model,dl,country,):

    R_base = get_from_trace("R_t_base",trace)
    R_soccer = get_from_trace("R_t_soccer",trace)
    C_base = get_from_trace("C_base",trace)
    C_soccer = get_from_trace("C_soccer",trace)
    
    # Calculate Effective R per sex
    R_eff = np.einsum('dt,dmf->dtmf',R_base, C_base) + np.einsum('dt,dmf->dtmf',R_soccer, C_soccer)
  

    
    fig,axes = plt.subplots(C_base.shape[1],C_base.shape[2],)
    covid19_uefa.plot.distribution(
        model=model,
        trace=trace,
        key="C_base",
        dist_math="C_{base}",
        ax=axes
    )
    
    fig.suptitle(country + f" - C_base")
    
    # Save figure as pdf and png        
    kwargs = {
        "transparent":True,
        "dpi":300,
        "bbox_inches":"tight"
    }
    fig.savefig(f"{fig_path}/fig_C_base_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_C_base_{country}.png", **kwargs)
    plt.close(fig=fig)
    

    fig,axes = plt.subplots(C_base.shape[1],C_base.shape[2],)
    covid19_uefa.plot.distribution(
        model=model,
        trace=trace,
        key="C_soccer",
        dist_math="C_{soccer}",
        ax=axes
    )
    
    # title
    fig.suptitle(country + f" - C_soccer")
    
    # Save figure as pdf and png        
    kwargs = {
        "transparent":True,
        "dpi":300,
        "bbox_inches":"tight"
    }
    fig.savefig(f"{fig_path}/fig_C_soccer_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_C_soccer_{country}.png", **kwargs)
    plt.close(fig=fig)
    
    
    fig, axes = plt.subplots(1,2,figsize=(3.4,2.7/2))
    
    covid19_uefa.plot.distribution(
        model=model,
        trace=trace,
        key="factor_female",
        dist_math="f_{female}",
        ax=axes[0]
    )
    covid19_uefa.plot.distribution(
        model=model,
        trace=trace,
        key="c_off",
        dist_math="c_{offdiagonal}",
        ax=axes[1]
    )
    fig.savefig(f"{fig_path}/fig_overviewContact_{country}.pdf", **kwargs)
    fig.savefig(f"{fig_path}/fig_overviewContact_{country}.png", **kwargs)
    plt.close(fig=fig)  
    
    

In [4]:
def load(fstr):
    with open(fstr, "rb") as f:
         return pickle.load(f)
        
        
xlims_fraction = {
    "Germany":[0.8,1.5],
    "Scotland":[0.8,2.1],
    "England":[0.8,1.25],
    "France":[0.8,1.25],
}
models,traces,dls = {},{},{}
for country in countries:
    # Load trace, model and dataloader
    fstr=f"../scripts/run_on_cluster/pickled/UEFA-beta=False-country={country}-tune=1000-draws=1500-max_treedepth=12.pickled"

    model, trace = load(fstr)
    dl = covid19_uefa.dataloader.Dataloader_gender(countries=[country])
    models[country]=model
    traces[country]=trace
    dls[country]=dl
    
    # Create first look plots for each country
    #plot_casesOverview(trace,model,dl,country,xlim_fraction=xlims_fraction[country])
    #plot_reproductionNumber(trace,model,dl,country)
    #plot_alpha(trace,model,dl,country)
    #plot_alpha(trace,model,dl,country)
    #plot_violin(trace,model,dl,country)
    #plot_violin(trace,model,dl,country,beta=True)
    #plot_contacts(trace, model,dl,country)
    

## Main plots

In [30]:
def plot_fraction(ax,trace,model,dl,xlim_fraction):
    
    new_cases = get_from_trace("new_cases",trace)
    
    ## Fraction male/female
    cov19.plot._timeseries(
        x = pd.date_range(model.sim_begin,model.sim_end),
        y = (new_cases[:,:,0]/dl.population[0,0]) / (new_cases[:,:,1]/dl.population[1,0]), # male/female
        what="model",
        ax=ax,
        color=colors_paul[3]
    )
    cov19.plot._timeseries(
        x = pd.date_range(model.data_begin,model.data_end),
        y = (dl.new_cases_obs[:,0,0]/dl.population[0,0])/(dl.new_cases_obs[:,1,0]/dl.population[1,0]), # male/female
        what="data",
        ax=ax,
        color=colors_paul[3],
        ms=2
    )

    ax.set_ylabel("Incidence\nsex ratio")
    ax.set_ylim(xlim_fraction)
    ax.set_xlim(xlim_ts)
    for label in ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)
        
    return ax

def plot_rsoccer(ax,trace,model,dl):
    """
    Plots the base and soccer reproduction number 
    
    Parameters
    ----------
    trace:
        arviz trace of model run
    model:
        corresponding model
    dl:
        dataloader
    """
    R_soccer = get_from_trace("R_t_soccer",trace)
    C_soccer = get_from_trace("C_soccer",trace)
    
    # Plot base and soccer Reproduction number
    cov19.plot._timeseries(
        x = pd.date_range(model.sim_begin,model.sim_end),
        y = R_soccer,
        what="model",
        ax=ax,
        color=colors_paul[4]
    )
    ax.axhline(0,color="tab:gray",ls="--",zorder=-5,lw=0.5)
    ax.set_ylabel("Additive\nrep. number")
    ax.set_ylim(-0.8,2.1)
    
    return ax

def plot_alphaMean(ax,trace,model,dl,beta=False):
    
    if not beta:
        alpha = get_from_trace(f"alpha_mean",trace)
        R_soccer = np.exp(alpha)-1
    else:
        try:
            alpha = get_from_trace(f"beta_mean",trace)
            R_soccer = np.exp(alpha)-1
        except:
            return ax
        
    import seaborn.categorical
    seaborn.categorical._Old_Violin = seaborn.categorical._ViolinPlotter
    
    sns.violinplot(
        data=R_soccer,
        scale="width",
        inner="quartile",
        orient="v",
        ax=ax,
        color=colors_paul[5],
        linewidth=1,
        saturation=1
    )
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.collections[0].set_edgecolor(colors_paul[5]) # Set outline colors
    ax.set_xticklabels([])
    ax.set_ylabel("Mean effect of all \n played soccer games")
    ax.axhline(0,color="tab:gray",ls="--",zorder=-5)
    return ax

In [31]:

xlims_fraction = {
    "Germany":[0.8,1.5],
    "Scotland":[0.8,2.1],
    "England":[0.9,1.35],
    "France":[0.75,1.25],
}
id2_country = np.array([["Germany","France"],["England","Scotland"]])


fig = plt.figure(figsize=(7.5,5))
# Two columns/rows
outer_grid = fig.add_gridspec(2, 2, wspace=0.35, hspace=0.25,)
# Two rows
axes = []
for a in range(2):
    for b in range(2):
        
        # gridspec inside gridspec
        inner_grid = outer_grid[a,b].subgridspec(2, 2, width_ratios=[1,0.3])
        
        # Create three subplots
        # - a1: fraction
        # - a2: R_soccer
        # - a3: alpha mean
        # - a4: beta mean
        country = id2_country[a,b]
        
        a1 = fig.add_subplot(inner_grid[0,0])
        plot_fraction(a1, traces[country], models[country], dls[country], xlims_fraction[country])
        
        
        a2 = fig.add_subplot(inner_grid[1,0])
        plot_rsoccer(a2, traces[country], models[country], dls[country])
        
        
        a3 = fig.add_subplot(inner_grid[0:,-1])
        plot_alphaMean(a3, traces[country], models[country], dls[country])

        #a4 = fig.add_subplot(inner_grid[0:,-1])
        #plot_alphaMean(a4, traces[country], models[country], dls[country],beta=True)

        # Markup
        a1.set_title(country)
        for ax in [a1,a2]:
            ax.set_xlim(xlim_ts)
            #Locator
            ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=4))
            ax.xaxis.set_minor_locator(mdates.WeekdayLocator(interval=1))
            # Hide the right and top spines
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
                
        a1.set(xticklabels=[])

        #a3.spines['right'].set_visible(False)
        a3.spines['top'].set_visible(False)
        a3.spines['bottom'].set_visible(False)
        a3.spines['left'].set_visible(False)
        a3.tick_params(bottom=False)
        
        if a == 1 and b == 1:
            a3.set_ylim(-2,12)
            a3.set(yticks=[0,4,8])
        else: 
            a3.set_ylim(-1,6)
            a3.set(yticks=[0,2,4])
        

fig.align_ylabels()
# Save figure as pdf and png        
kwargs = {
    "transparent":True,
    "dpi":300,
    "bbox_inches":"tight"
}
fig.savefig(f"{fig_path}/fig_1.pdf", **kwargs)
fig.savefig(f"{fig_path}/fig_1.png", **kwargs)
plt.close(fig=fig)

In [40]:
## P-value
import arviz as az
az.plot_bpv(traces["England"],group="prior")
plt.show()

TypeError: `data` argument must have the group "prior_predictive"

In [44]:
import pymc3 as pm
key= "new_cases"
prior = pm.sample_prior_predictive(
    samples=1000, model=models["England"], var_names=["R_t_soccer"]
)



In [48]:
traces["England"] + az.convert_to_inference_data(prior,group="prior_predictive")

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> prior_predictive
	> observed_data

Warmup iterations saved (warmup_*).